# Contents
3.1 [Displaying Row Numbers](#3.1-Displaying-Row-Numbers) <br>
3.2 [Using the FORMAT= Column Modifier to Format Output](#3.2-Using-the-FORMAT=-Column-Modifier-to-Format-Output) <br>
3.3 [Concatenating Character Strings](#3.3-Concatenating-Character-Strings) <br>
3.4 [Using Scalar Expressions with Selected Columns](#3.4-Using-Scalar-Expressions-with-Selected-Columns) <br>
3.5 [Ordering Output by Column](#3.5-Ordering-Output-by-Columns) <br>
3.6 [Grouping Data with Summary Functions](#3.6-Grouping-Data-with-Summary-Functions) <br>
3.7 [Subsetting Groups with the HAVING Clause](#3.7-Subsetting-Groups-with-the-HAVING-Clause)

In [1]:
import saspy               # Import the saspy module
sas = saspy.SASsession()   # Connect to the SAS environment

Using SAS Config named: oda
SAS Connection established. Subprocess id is 27900



In [2]:
%%SAS
libname sql '~/sql';   *Assign a library reference;
title;                 *No title for all procedures output;

Using SAS Config named: oda
SAS Connection established. Subprocess id is 18576



### 3.1 Displaying Row Numbers
You can specify an SQL procedure option called `NUMBER` to display row numbers on output under the column heading Row. As with the Obs column produced by the print procedure, the NUMBER option displays row numbers on output. 

In [3]:
%%SAS
proc sql number;
    select *
        from sql.invoice
quit;

Row,Invoice Number,Manufacturer Number,Customer Number,Invoice Quantity - Units Sold,Invoice Unit Price,Product Number
1,1001,500,201,5,"$1,495.00",5001
2,1002,600,1301,2,"$1,598.00",6001
3,1003,210,101,7,$245.00,2101
4,1004,111,501,3,"$9,600.00",1110
5,1005,500,801,2,$798.00,5002
6,1006,500,901,4,$396.00,6000
7,1007,500,401,7,"$23,100.00",1200


### 3.2 Using the FORMAT= Column Modifier to Format Output
The SQL procedure supports the specification of a `FORMAT=` column modifier for purposes of providing instructions on how to write character and numeric data values by a query expression. The general syntax for writing a format follows: <br>
`<$>format<w>.<d>`

`$` is used to reference a character format and its absence references a numeric format. <br>
`w` references the format width, which is typically represented by the number of output columns used for the data. <br>
`d` represents the optional decimal positioning for numeric data. 

In [4]:
%%SAS
proc sql;    
    select prodname,
           prodtype,
           prodcost * 0.80 as Discount_Price format=dollar9.2
        from sql.products
            order by 3;
quit;

Product Name,Product Type,Discount_Price
Analog Cell Phone,Phone,$28.00
Office Phone,Phone,$104.00
Digital Cell Phone,Phone,$140.00
Spreadsheet Software,Software,$239.20
Graphics Software,Software,$239.20
Wordprocessor Software,Software,$239.20
Database Software,Software,$319.20
Dream Machine,Workstation,"$2,560.00"
Business Machine,Workstation,"$2,640.00"
Travel Laptop,Laptop,"$2,720.00"


### 3.3 Concatenating Character Strings
Two or more strings can be concatenated to produce a combined and longer string of characters. The concatenation character string operator, which is represented by two vertical bars "`||`", combines two or more strings or columns together to form a new string value. 

In [5]:
%%SAS
*The two SQL procedures produce the same resulting set;
proc sql;
    select trim(manucity) || ', '|| manustat as Headquarters 
            	/* The TRIM function removes trailing blanks */
        from sql.manufacturers;
quit;

proc sql;
    select catx(', ', manucity, manustat) as Headquarters
            /* The CATX function removes leading and trailing blanks of character strings
               and concatenates the strings by a specified separator */
        from sql.manufacturers;
quit;

Headquarters
"Houston, TX"
"San Diego, CA"
"Miami, FL"
"San Mateo, CA"
"San Diego, CA"
"San Diego, CA"
Headquarters
"Houston, TX"
"San Diego, CA"
"Miami, FL"


### 3.4 Using Scalar Expressions with Selected Columns
In computing terms, a **scalar** refers to a quantity represented by a single number of value. PROC SQL allows the use of scalar expression and constants with selected columns. 

In [6]:
%%SAS
proc sql;
    select prodname,
           prodcost,
           .075 * prodcost as Sales_Tax 
                format=dollar7.2 label='Sales Tax'
        from sql.products;
quit;

Product Name,Product Cost,Sales Tax
Dream Machine,"$3,200.00",$240.00
Business Machine,"$3,300.00",$247.50
Travel Laptop,"$3,400.00",$255.00
Analog Cell Phone,$35.00,$2.63
Digital Cell Phone,$175.00,$13.13
Office Phone,$130.00,$9.75
Spreadsheet Software,$299.00,$22.43
Database Software,$399.00,$29.93
Wordprocessor Software,$299.00,$22.43
Graphics Software,$299.00,$22.43


### 3.5 Ordering Output by Columns
By definition, **tables** are unordered sets of data. `ORDER BY` clause can be used to arrange rows of data in ascending or descending (`DESC`) order. For each column specified in the clause, ascending is the default order.  

In [7]:
%%SAS
*The resulting set is arranged first in ascending order by product type (PRODTYPE)
and then within product type in descending order by product cost (PRODCOST);
proc sql;
    select prodname, prodtype, prodcost, prodnum
        from sql.products
            order by prodtype, prodcost desc;
quit;

Product Name,Product Type,Product Cost,Product Number
Travel Laptop,Laptop,"$3,400.00",1700
Digital Cell Phone,Phone,$175.00,2102
Office Phone,Phone,$130.00,2200
Analog Cell Phone,Phone,$35.00,2101
Database Software,Software,$399.00,5002
Spreadsheet Software,Software,$299.00,5001
Graphics Software,Software,$299.00,5004
Wordprocessor Software,Software,$299.00,5003
Business Machine,Workstation,"$3,300.00",1200
Dream Machine,Workstation,"$3,200.00",1110


### 3.6 Grouping Data with Summary Functions
Occasionally it may be useful to display data in designated groups. When a `GROUP BY` is used with a summary function, the rows are aggregated in a series of groups. This means that an aggregate function is evaluated on a group of rows and not on a single row at a time. 

In [8]:
%%SAS
*Determine the least expensive product in each product type category
 and arrange the resulting set by the least expensive product;
proc sql;
    select prodtype,
           min(prodcost) as Cheapest
                   format=dollar9.2 label='Least Expensive'
        from sql.products
            group by prodtype
                order by Cheapest;
quit;

Product Type,Least Expensive
Phone,$35.00
Software,$299.00
Workstation,"$3,200.00"
Laptop,"$3,400.00"


### 3.7 Subsetting Groups with the HAVING Clause
When processing groups of data, it is frequently useful to subset aggregated rows (or groups) of data. SQL provides a convenient way to subset (or filter) groups of data by using the GROUP BY clause and the `HAVING` clause. The HAVING clause is optional, but when specified is used in combination with the GROUP BY clause. 

In [9]:
%%SAS
*Identify those software and hardware products that have an average cost 
 that is less than $400.00 from the PRODUCTS table;
proc sql;
    select prodtype,
           avg(prodcost) as Average_Product_Cost
                   format=dollar12.2
        from sql.products
            where prodtype in ('Software', 'Hardware')
                group by prodtype
                    having Average_Product_Cost < 400;
quit;

Product Type,Average_Product_Cost
Software,$324.00
